In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('test.csv')
df.__delitem__('id')

In [3]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0
1,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0
2,1807,1,2.8,0,1,0,27,0.9,186,3,4,1270,1366,2396,17,10,10,0,1,1
3,1546,0,0.5,1,18,1,25,0.5,96,8,20,295,1752,3893,10,0,7,1,1,0
4,1434,0,1.4,0,11,1,49,0.5,108,6,18,749,810,1773,15,8,7,1,0,1


In [5]:
X = df.drop(('dual_sim'), axis=1)
y = df['dual_sim']
feature_names = X.columns
print (feature_names)

Index(['battery_power', 'blue', 'clock_speed', 'fc', 'four_g', 'int_memory',
       'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram',
       'sc_h', 'sc_w', 'talk_time', 'three_g', 'touch_screen', 'wifi'],
      dtype='object')


In [6]:
print (X.shape)
print (y.shape)
N, d = X.shape

(1000, 19)
(1000,)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 43)

first_train, _ = X_train.shape
second_train, _ = X_test.shape 
print (first_train, second_train)

700 300


In [22]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

bagging = BaggingClassifier(DecisionTreeClassifier(max_depth=25, random_state=99), max_samples=1.0, max_features=2.0)
bagging.fit(X_train, y_train)
predict0 = bagging.predict(X_test)
pred_b = bagging.predict_proba(X_test)

acc_b = metrics.accuracy_score(y_test,  predict0)
print (acc_b)

0.5


In [9]:
roc_b = metrics.roc_auc_score(y_test, predict0)
print(roc_b)

0.5159139402560455


In [10]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier(max_depth=25 ,n_estimators=100, random_state=99)
rf.fit(X_train, y_train)
predict_rf = rf.predict(X_test)

acc_r = metrics.accuracy_score(y_test,  predict_rf)
print(acc_r)

0.51


In [11]:
roc_r = metrics.roc_auc_score(y_test, predict_rf)
print(roc_r)

0.5103129445234709


In [12]:
from sklearn.linear_model import SGDClassifier

classifier = SGDClassifier(loss='log',random_state=2, max_iter = 1000)
classifier.fit(X_train, y_train)
predict_cl = classifier.predict(X_test)
pred_cl = classifier.predict_proba(X_test)

acc_c = metrics.accuracy_score(y_test,  predict_cl)
print(acc_c)

0.51


C:\Users\worlak\PycharmProjects\ds_school\venv\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\worlak\PycharmProjects\ds_school\venv\lib\site-packages\sklearn\linear_model\base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [13]:
roc_c = metrics.roc_auc_score(y_test, predict_cl)
print(roc_c)

0.503645092460882


In [14]:
from sklearn import neighbors

knn = neighbors.KNeighborsClassifier(n_neighbors=17)
knn.fit(X_train, y_train)
predict_k = knn.predict(X_test)
pred_k = knn.predict_proba(X_test)

acc_k = metrics.accuracy_score(y_test, predict_k)
print(acc_k)

0.49666666666666665


In [15]:
roc_k = metrics.roc_auc_score(y_test, predict_k)
print(roc_k)

0.49804409672830724


In [16]:
per_b = (acc_b + roc_b) / 2
per_r = (acc_r + roc_r) / 2
per_c = (acc_c + roc_c) / 2
per_k = (acc_k + roc_k) / 2

In [17]:
t = [per_b, per_r, per_c, per_k]
t = max(t)

In [18]:
if per_b == t:
    print ('Bagging Classifier accuracy score:', acc_b, 'roc auc score:', roc_b)
elif per_r == t:
    print ('Random Forest Classifier accuracy score:', acc_r, 'roc auc score:', roc_r)
elif per_c == t:
    print ('Linear classifier accuracy score:', acc_c, 'roc auc score:', roc_c)
elif per_k == t:
    print ('K Neighbors Classifier accuracy score:', acc_k, 'roc auc score:', roc_k)

Bagging Classifier accuracy score: 0.5166666666666667 roc auc score: 0.5159139402560455


In [19]:
from sklearn.ensemble import VotingClassifier

vot = VotingClassifier(estimators=[('lr', classifier), ('rf', rf), ('bg', bagging), ('kn', knn)], voting='hard')

vot.fit(X_train, y_train)
predict_v = vot.predict(X_test)

roc_v = metrics.roc_auc_score(y_test, predict_v)
print (roc_v)

C:\Users\worlak\PycharmProjects\ds_school\venv\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


0.5091571834992888


In [20]:
acc_v = metrics.accuracy_score(y_test, predict_v)
print (acc_v)

0.5133333333333333
